In [1]:
import pandas as pd
import re
import datetime
import numpy as np

# 2019 Data Preprocess

In [2]:
pbp_data_19 = pd.read_csv("./pbp_data/regular_season/reg_pbp_2019.csv")

C:\Users\costi\AppData\Local\Temp\ipykernel_25664\20673864.py:1: DtypeWarning: Columns (167,168,169,170,175,176,179,180,183,184,189,190,191,192,195,196,204,205,206,219,220,221,222,223,224,227,228,232,233,234,239,241,242) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp_data_19 = pd.read_csv("./pbp_data/regular_season/reg_pbp_2019.csv")


In [3]:
highlights19 = pd.read_csv("./2019_NFL_highlight_transcript - Sheet1.csv")
highlights19 = highlights19[~highlights19["inning"].isna()]

In [4]:
relevant_cols = pbp_data_19.drop(columns=(list(pbp_data_19.iloc[:,58:110].columns) + ["desc","home_timeouts_remaining","home_timeouts_remaining","timeout","timeout_team","play_id","game_id","game_date"])).columns

In [5]:
pbp_data_19.columns

Index(['play_id', 'game_id', 'home_team', 'away_team', 'posteam',
       'posteam_type', 'defteam', 'side_of_field', 'yardline_100', 'game_date',
       ...
       'penalty_player_id', 'penalty_player_name', 'penalty_yards',
       'replay_or_challenge', 'replay_or_challenge_result', 'penalty_type',
       'defensive_two_point_attempt', 'defensive_two_point_conv',
       'defensive_extra_point_attempt', 'defensive_extra_point_conv'],
      dtype='object', length=256)

In [6]:
def process_inning_time(time_str):
    quarter = time_str[0]
    if quarter == "O":
        time_remaining = 0
    else:
        time_remaining = (4 - int(quarter)) * 900
    times = re.findall("\d?\d:\d\d",time_str)
    timedeltas = pd.to_timedelta(pd.to_datetime(times,format='%M:%S').strftime('%H:%M:%S').values)
    timelists = (timedeltas + datetime.timedelta(seconds=time_remaining)).seconds
    return pd.Interval(timelists[1],timelists[0],closed="both")

In [7]:
highlights19["intervals"] = highlights19["inning"].apply(process_inning_time)
highlights19["date"] = pd.to_datetime(highlights19["date"])
pbp_data_19["game_date"] = pd.to_datetime(pbp_data_19["game_date"])

In [8]:
merged19 = pbp_data_19.merge(highlights19,left_on=["home_team","away_team","game_date"], right_on=["home","away","date"])

In [9]:
def in_interval(row):
    return row["game_seconds_remaining"] in row["intervals"]

In [10]:
plays19 = merged19[merged19.apply(lambda x: x["game_seconds_remaining"] in x["intervals"],axis=1)]

In [11]:
X = plays19[relevant_cols.to_list()+["transcript"]]

In [12]:
commentary = pd.Series(plays19["transcript"].unique()).sample(3).values

In [13]:
# commentary[0]

In [14]:
my_count = -1
while my_count < 10:
    commentary = pd.Series(plays19["transcript"].unique()).sample(10).values
    my_count = X.loc[X["transcript"].isin(commentary),"touchdown"].sum()

In [15]:
commentary

array(['Cowboy 28, little trickery! and a pass to boot. wide open, single terry with a buffalo touchdown!',
       'You just have to adapt as the season continues to go along. From the 15 to the end zone and it is caught for a touchdown by dez bryant oh van tell bryant!',
       'mayfield throws wide open! to the five! to the end zone! ricky seals jones with the touchdown!',
       'the second and goal, Jones in the backfield. Winston, quick release! Taking it in for his second touchdown in this quarter is Mike Evans!',
       'Wilson steps through, flips wide open and caught! After a juggle for the touchdown!  Carson!',
       'first and 10 25 yard line off the fake to goldman! Jones pass is caught by Evan Engram still going! Down the sideline! Evan Edward will take this all the way! What a great read by Daniel Jones! More importantly, what a great read by Evan Engram!',
       'First and 15. Chubb. Chubb with running room to the 40! On his way to the end zone!',
       "Second and go

In [16]:
tempX = X.loc[X["transcript"].isin(commentary)]

In [17]:
transcript_grouped = tempX.groupby("transcript").groups
transcript_groups = list(transcript_grouped.keys())

In [18]:
plays19.loc[transcript_grouped[transcript_groups[4]]][["inning","URL"]]

,inning,URL
10685,1st inning 10:40-10:34,https://www.youtube.com/watch?v=hmVut-6Acp4
10695,1st inning 10:40-10:34,https://www.youtube.com/watch?v=hmVut-6Acp4
10705,1st inning 10:40-10:34,https://www.youtube.com/watch?v=hmVut-6Acp4


In [19]:
transcript_groups[4]

"Second and goal, play action, prescott finds the target it's waiting for the touchdown! Jason Witten has another thanksgiving touchdown!"

In [20]:
transcript_groups

['Cowboy 28, little trickery! and a pass to boot. wide open, single terry with a buffalo touchdown!',
 'First and 15. Chubb. Chubb with running room to the 40! On his way to the end zone!',
 "First snap inside the 20. allen open field inside the 10 and he goes all the way for the touchdown! He's got yet another game with a touchdown pass and a touchdown run",
 'Nick Chubb trying ot find the endzone and he does! Touchdown Cleveland.',
 "Second and goal, play action, prescott finds the target it's waiting for the touchdown! Jason Witten has another thanksgiving touchdown!",
 'Wilson steps through, flips wide open and caught! After a juggle for the touchdown!  Carson!',
 'You just have to adapt as the season continues to go along. From the 15 to the end zone and it is caught for a touchdown by dez bryant oh van tell bryant!',
 'first and 10 25 yard line off the fake to goldman! Jones pass is caught by Evan Engram still going! Down the sideline! Evan Edward will take this all the way! What

In [21]:
ret_list = []
for i in range(len(transcript_groups)):
    transcript_group = X.loc[transcript_grouped[transcript_groups[i]]]
    ret_list += [{"json":transcript_group.loc[transcript_group["touchdown"]==1,relevant_cols].replace(0,np.nan).dropna(axis=1).iloc[0].to_json(),
                  "transcript":transcript_group["transcript"].iloc[0]}]

In [24]:
ret_list[4]["transcript"]

"Second and goal, play action, prescott finds the target it's waiting for the touchdown! Jason Witten has another thanksgiving touchdown!"